In [1]:
import cv2
import numpy as np

In [2]:
# Cargar la red YOLO preentrenada
net = cv2.dnn.readNet('./../../../venv/yolov3.weights', './../../../venv/yolov3.cfg')
layer_names = net.getUnconnectedOutLayersNames()

cap = cv2.VideoCapture(0)

In [4]:


while True:
    ret, frame = cap.read()

    # Obtener las dimensiones de la imagen
    height, width, _ = frame.shape

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Obtener las detecciones
    detections = net.forward(layer_names)

    # Procesar las detecciones
    boxes = []
    confidences = []

    for detection in detections[0]:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > 0.5 and class_id == 2:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Coordenadas de la esquina superior izquierda
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))

    # Aplicar NMS
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Dibujar rectángulo solo para las detecciones después de NMS
    for i in indices:
        x, y, w, h = boxes[i]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Mostrar la imagen resultante
    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [12]:
import cv2
import numpy as np

# Cargar la red YOLO
net = cv2.dnn.readNet('./../../../venv/matriculas.weights', './../../../venv/matriculas.cfg')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
layer_names = net.getUnconnectedOutLayersNames()

# Configurar la captura de video desde la cámara (puedes cambiar 0 a la ID de tu cámara)
cap = cv2.VideoCapture(1)

while True:
    # Capturar el fotograma de la cámara
    ret, frame = cap.read()

    if not ret:
        break

    height, width = frame.shape[:2]

    # Escalar y normalizar la imagen
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Obtener las detecciones
    outs = net.forward(layer_names)

    class_ids = []
    confidences = []
    boxes = []

    # Umbral de confianza para las detecciones
    conf_threshold = 0.5

    # Umbral de supresión no máxima
    nms_threshold = 0.4

    # Recorrer las detecciones y filtrar aquellas con confianza suficiente
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Coordenadas del cuadro delimitador
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Aplicar supresión no máxima para evitar detecciones duplicadas
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # Dibujar cuadro verde alrededor de la matrícula
    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Mostrar la imagen con los cuadros delimitadores
    cv2.imshow('Matricula Detectada', frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la captura de video y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()


IndexError: invalid index to scalar variable.